In [1]:
import pandas as pd
import numpy as np
import joblib

In [2]:
import os
parent_directory = os.path.dirname(os.getcwd())
os.chdir(parent_directory)
os.getcwd()

'd:\\When-Hybrid-meets-the-Prompt'

In [3]:
from Model.Prompt import prompt

In [4]:
df_courses = pd.read_csv("Data/courses.csv", index_col=0).dropna().drop_duplicates()
df_courses = df_courses.reset_index()
df_courses.head()

,index,item_id,item_name,item_avg_rating,item_category,item_subcategory,item_members,item_description
0,0,473160,Web Design for Web Developers: Build Beautiful...,4.466391,Development,Web Development,681494.0,IMPORTANT NOTE: The material of this course is...
1,1,433798,Introduction To Python Programming,4.408828,Development,Programming Languages,885914.0,Are you interested in becoming a programmer? P...
2,2,5340996,Python Interview Preparation Coding Exercises,3.596154,Development,Programming Languages,11357.0,Level Up Your Python Skills for Coding Intervi...
3,3,53600,Useful Excel for Beginners,4.512998,Office Productivity,Microsoft,603913.0,\t Welcome to the Useful Excel for Beginn...
4,4,24823,Java Tutorial for Complete Beginners,4.374596,Development,Programming Languages,1840631.0,\t Learn to program in the Java progr...


In [5]:
vectors = joblib.load("Assets/courses_embeddings.pkl")

In [6]:
def get_rec():
    rss = []
    for i in rec_prompt[:10]:
        try:
            rec_title = df_courses.iloc[i[0]]["item_name"]
            rec_avg_rating = df_courses.iloc[i[0]]["item_avg_rating"]
            rec_category = df_courses.iloc[i[0]]["item_category"]

            rs = {
                'name': rec_title,
                'rating': rec_avg_rating,
                'category': rec_category
            }
            # print(rs)
            rss.append(rs)
            print(rec_title,rec_avg_rating,rec_category)
        except:
            continue
    return rss

đầu tiên là lấy input từ user, sau đó sử dụng `SentenceTransformer()` encode input, có embeddings của input đó, đối số vectors là các courses embeddings có được cũng bằng cách sử dụng ```SentenceTransformer()``` rồi encode. 

sau đó tính toán similarity của input embs và courses embs bằng ```consine_similarity()``` của thư viện sklearn

similarity là sự tương tự của user input so với từng courses -> sort similarity in descending order ```reverse = True``` và lấy top k item
```enumerate()``` : chuyến similarity sang dạng (index, element)
```list()```: chuyển sang list
```sorted( , reverse = True, key = )```: sort in descending order (reverse) dựa trên similarity scores


In [7]:
input_prompt_1 = "Im currently majoring in Digital Business and Artifiical Intelligent, i want to focus in ML algorithms"
prompt_1 = prompt.RecommenderPrompt(df=df_courses,
                               top_k=5,
                               vectors=vectors,
                               input_prompt=input_prompt_1)
prompt_1.recCourses()
rec_prompt = prompt_1.rec_rs

output_1 = get_rec()

Red Hat Enterprise Linux Technical Overview 4.5 IT & Software
Fundamental Practices of Content Creation for Marketers 4.4 Marketing
Make Photoshop Brushes, Vectors, & Repeating Patterns. 4.8 Design
CodePen Editor Clone: Powerful Code Editor Built with React. 4.7777777 Development
Modern Data Visualization with Oracle Analytics Cloud 4.3333335 Business
Finance Fundamentals Crash Course: Time Value of Money 4.8125 Finance & Accounting
Engineering Materials 4.5 Teaching & Academics
Quick & Easy to Make Video: YouTube Shorts, Instagram Reels 3.75 Photography & Video
IT Management: Communication and Motivation 3.8 Business


In [8]:
input_prompt_2 = "I want to be a designer"
prompt_2 = prompt.RecommenderPrompt(df=df_courses,
                               top_k=5,
                               vectors=vectors,
                               input_prompt=input_prompt_2)
prompt_2.recCourses()
rec_prompt = prompt_2.rec_rs

output_2 = get_rec()

Professional Fully Accredited NLP Certification Diploma 4.75 Personal Development
R, ggplot, and Simple Linear Regression 4.6904764 Development
Analyse Garment Construction 4.25 Personal Development
Productivity Booster 4.1 Personal Development
Engineering Calculations using Microsoft Excel 4.05 Office Productivity
How to Start a Blog from Scratch 4.05 Marketing
Learn Oracle 11g  - A Comprehensive Training Course 4.15 Office Productivity
Color Basics for Print Designers 4.3 Design
Basic of C++ Programming Language 3.6666667 Development
Youtube SEO Course To Rank On First Page : Views & Rank Hack 4.25 Marketing


In [9]:
input_prompt_3 = "I'm looking for courses that offer critical thinking skill, I want to improve my skills in leadership"
prompt_3 = prompt.RecommenderPrompt(df=df_courses,
                               top_k=5,
                               vectors=vectors,
                               input_prompt=input_prompt_3)
prompt_3.recCourses()
rec_prompt = prompt_3.rec_rs

output_3 = get_rec()

Introduction to the New SAP Fit-to-Standard design process 4.8 IT & Software
Foundation 5 for Beginners 3.8 Development
Fire Fighting System Design Basics 4.55 Design
How to Read People's Personality in Minutes using Psychology 4.111111 Teaching & Academics
Intelligent Automation (IA) Simplified - cheat sheets 4.7 Development
Advanced Adobe Acrobat XI 4.9 Design
Analytics For All 4.35 Business
From Procrastination To Productivity 4.388889 Personal Development
Adobe Illustrator-CC Logo & Graphic Design Crash Course 4.62 Design
Mechatronics and Industrial Internet of Things 4.0416665 Teaching & Academics


In [10]:
from bert_score import BERTScorer
from rouge import Rouge

In [11]:
def get_rouge_score(input, output):
    rouge = Rouge()
    return rouge.get_scores(input,output)

eval_1_rouge = get_rouge_score(input_prompt_1, str(output_1))
eval_2_rouge = get_rouge_score(input_prompt_2, str(output_2))
eval_3_rouge = get_rouge_score(input_prompt_3, str(output_3))

rouge_scores_out = []

for metric in ["rouge-1", "rouge-2", "rouge-l"]:
    for label in ["F-Score"]:
        eval_1_score = eval_1_rouge[0][metric][label[0].lower()]
        eval_2_score = eval_2_rouge[0][metric][label[0].lower()]
        eval_3_score = eval_3_rouge[0][metric][label[0].lower()]

        row = {
            "Metric": f"{metric} ({label})",
            "Prompt 1": eval_1_score,
            "Prompt 2": eval_2_score,
            "Prompt 3": eval_3_score,
        }
        rouge_scores_out.append(row)

rouge_scores_out = (
    pd.DataFrame(rouge_scores_out)
    .set_index("Metric")
)

rouge_scores_out

,Prompt 1,Prompt 2,Prompt 3
Metric,,,
rouge-1 (F-Score),0.041237,0.045977,0.061224
rouge-2 (F-Score),0.000000,0.000000,0.000000
rouge-l (F-Score),0.041237,0.045977,0.061224


In [12]:

# Instantiate the BERTScorer object for English language
scorer = BERTScorer(lang="en")

# Calculate BERTScore for the summary 1 against the excerpt
# P1, R1, F1_1 represent Precision, Recall, and F1 Score respectively
P1, R1, F1_1 = scorer.score([input_prompt_1], [str(output_1)])

# Calculate BERTScore for summary 2 against the excerpt
# P2, R2, F2_2 represent Precision, Recall, and F1 Score respectively
P2, R2, F2_2 = scorer.score([input_prompt_2], [str(output_2)])
P3, R3, F3_3 = scorer.score([input_prompt_3], [str(output_3)])

print("Prompt 1 F1 Score:", F1_1.tolist()[0])
print("Prompt 2 F1 Score:", F2_2.tolist()[0])
print("Prompt 3 F1 Score:", F3_3.tolist()[0])

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Prompt 1 F1 Score: 0.7554874420166016
Prompt 2 F1 Score: 0.7673749923706055
Prompt 3 F1 Score: 0.7710221409797668


In [13]:
print("Prompt 1 Precious Score:", P1.tolist()[0])
print("Prompt 2 Precious Score:", P2.tolist()[0])
print("Prompt 3 Precious Score:", P3.tolist()[0])

Prompt 1 Precious Score: 0.8003475666046143
Prompt 2 Precious Score: 0.8318243026733398
Prompt 3 Precious Score: 0.8281808495521545


In [14]:
print("Prompt 1 Recall Score:", R1.tolist()[0])
print("Prompt 2 Recall Score:", R2.tolist()[0])
print("Prompt 3 Recall Score:", R3.tolist()[0])

Prompt 1 Recall Score: 0.7153893709182739
Prompt 2 Recall Score: 0.7121945023536682
Prompt 3 Recall Score: 0.7212438583374023


In [15]:
# from llama_cpp import Llama
# llm = Llama(model_path="./models/7B/llama-model.gguf")
# output = llm("Q: Name the planets in the solar system? A: ", max_tokens=32, stop=["Q:", "\n"], echo=True)
# print(output)

In [16]:
# from langchain.llms import LlamaCpp
# from langchain.prompts import PromptTemplate
# from langchain.chains import LLMChain
# from langchain.callbacks.manager import CallbackManager
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [17]:
# # Callbacks support token-wise streaming
# callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [18]:
# from transformers import AutoTokenizer
# import transformers
# import torch
# import accelerate

# model = "meta-llama/Llama-2-7b-chat-hf"

# tokenizer=AutoTokenizer.from_pretrained(model)
# pipeline=transformers.pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     torch_dtype=torch.bfloat16,
#     trust_remote_code=True,
#     device_map="auto",
#     max_length=1000,
#     do_sample=True,
#     top_k=10,
#     num_return_sequences=1,
#     eos_token_id=tokenizer.eos_token_id
#     )


In [19]:
# from langchain.document_loaders import CSVLoader

In [20]:
# loader = CSVLoader("Data/courses.csv", encoding="utf8")
# data = loader.load()

In [21]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

# chunk_size = 2000
# chunk_overlap = 0

# r_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=chunk_size,
#     chunk_overlap=chunk_overlap
# )
# c_splitter = CharacterTextSplitter(
#     chunk_size=chunk_size,
#     chunk_overlap=chunk_overlap
# )

In [22]:
# docs = c_splitter.split_documents(data)
# texts = [item['text'].decode('utf-8') for item in docs]

In [23]:
# from langchain.text_splitter import TokenTextSplitter

# text_splitter = TokenTextSplitter(chunk_size=1, chunk_overlap=0)

# text1 = input_prompt_1
# text_splitter.split_text(text1)
# # ['foo', ' bar', ' b', 'az', 'zy', 'foo']